In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load_Data

In [ ]:
import pandas as pd

In [ ]:
data_review_MI = pd.read_csv('/content/drive/MyDrive/My_DS_FT_1/data_review_MI.csv')
data_review_DM = pd.read_csv('/content/drive/MyDrive/My_DS_FT_1/data_review_DM.csv')

#Musical Instruments

In [ ]:
data_review_MI.head()

,Reviewer_ID,asin,Reviewer_Name,Rating,Unix_Review_Time,Review_Time,Review_Likes,Review_Dislikes,Rating_Category,review_text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...",5,1393545600,2014-02-28,0,0,good,"good Not much to write about here, but it does..."
1,A14VAT5EAX3D9S,1384719342,Jake,5,1363392000,2013-03-16,13,1,good,Jake The product does exactly as it should and...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""",5,1377648000,2013-08-28,1,0,good,It Does The Job Well The primary job of this d...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""",5,1392336000,2014-02-14,0,0,good,GOOD WINDSCREEN FOR THE MONEY Nice windscreen ...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,5,1392940800,2014-02-21,0,0,good,No more pops when I record my vocals. This pop...


### Import Libraries

In [ ]:
import sys  
!{sys.executable} -m pip install contractions

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
import contractions
from contractions import contractions_dict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
raw_tokens=len([w for t in (data_review_MI["review_text"].apply(word_tokenize)) for w in t])
print('Number of raw tokens: {}'.format(raw_tokens))

Number of raw tokens: 1107055


In [ ]:
import re, string, unicodedata
import nltk
import contractions
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import inflect

### Pre-Processing Requisites

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text



# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stopword_list= stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [ ]:
def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    #sample = expand_contractions(sample)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

### Text Normalizer

In [ ]:
data_review_MI['Clean_Text'] = data_review_MI['review_text'].map(lambda text: normalize_and_lemmaize(text))

### Expanding Contractions

In [ ]:
data_review_MI['no_contract'] = data_review_MI['Clean_Text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
data_review_MI['Cleaned_Text'] = [' '.join(map(str, l)) for l in data_review_MI['no_contract']]
data_review_MI.head()

,Reviewer_ID,asin,Reviewer_Name,Rating,Unix_Review_Time,Review_Time,Review_Likes,Review_Dislikes,Rating_Category,review_text,Clean_Text,no_contract,Cleaned_Text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...",5,1393545600,2014-02-28,0,0,good,"good Not much to write about here, but it does...",good not much write exactly suppose filter pop...,"[good, not, much, write, exactly, suppose, fil...",good not much write exactly suppose filter pop...
1,A14VAT5EAX3D9S,1384719342,Jake,5,1363392000,2013-03-16,13,1,good,Jake The product does exactly as it should and...,jake product exactly quite affordablei not rea...,"[jake, product, exactly, quite, affordablei, n...",jake product exactly quite affordablei not rea...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""",5,1377648000,2013-08-28,1,0,good,It Does The Job Well The primary job of this d...,job well primary job device block breath would...,"[job, well, primary, job, device, block, breat...",job well primary job device block breath would...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""",5,1392336000,2014-02-14,0,0,good,GOOD WINDSCREEN FOR THE MONEY Nice windscreen ...,good windscreen money nice windscreen protect ...,"[good, windscreen, money, nice, windscreen, pr...",good windscreen money nice windscreen protect ...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,5,1392940800,2014-02-21,0,0,good,No more pops when I record my vocals. This pop...,no pop record vocals pop filter great look per...,"[no, pop, record, vocals, pop, filter, great, ...",no pop record vocals pop filter great look per...


In [ ]:
data_review_MI = data_review_MI.drop(['no_contract','Clean_Text'], axis = 1)
data_review_MI.head()

,Reviewer_ID,asin,Reviewer_Name,Rating,Unix_Review_Time,Review_Time,Review_Likes,Review_Dislikes,Rating_Category,review_text,Cleaned_Text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...",5,1393545600,2014-02-28,0,0,good,"good Not much to write about here, but it does...",good not much write exactly suppose filter pop...
1,A14VAT5EAX3D9S,1384719342,Jake,5,1363392000,2013-03-16,13,1,good,Jake The product does exactly as it should and...,jake product exactly quite affordablei not rea...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""",5,1377648000,2013-08-28,1,0,good,It Does The Job Well The primary job of this d...,job well primary job device block breath would...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""",5,1392336000,2014-02-14,0,0,good,GOOD WINDSCREEN FOR THE MONEY Nice windscreen ...,good windscreen money nice windscreen protect ...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,5,1392940800,2014-02-21,0,0,good,No more pops when I record my vocals. This pop...,no pop record vocals pop filter great look per...


In [ ]:
# Let's put aside number of raw tokens in order to measure of cleaned tokens
from nltk.tokenize import word_tokenize
clean_tokens=len([w for t in (data_review_MI["Cleaned_Text"].apply(word_tokenize)) for w in t])

print('Number of clean tokens: {}\n'.format(clean_tokens))
print('Percentage of removed tokens: {0:.2f}'.format(1-(clean_tokens/raw_tokens)))

Number of clean tokens: 516322

Percentage of removed tokens: 0.53
